In [1]:
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar

In [2]:
rng = da.random.default_rng()
X = rng.standard_normal((3, 1000, 1000, 1000), chunks=(-1, -1, 100, 100))
X

dask.array<standard_normal, shape=(3, 1000, 1000, 1000), dtype=float64, chunksize=(3, 1000, 100, 100), chunktype=numpy.ndarray>

In [3]:
%run TC/EC_function.ipynb

In [4]:
dask_graph = X.map_blocks(ec_covar_multi, corr_sets=[1,2,3], dtype=np.float64,
                          chunks=(X.chunksize[0], X.chunksize[0]) + X.chunksize[2:],
                          drop_axis=[0, 1], new_axis=[0, 1])
dask_graph

dask.array<ec_covar_multi, shape=(3, 3, 1000, 1000), dtype=float64, chunksize=(3, 3, 100, 100), chunktype=numpy.ndarray>

In [5]:
#dask_graph.visualize()

In [6]:
with ProgressBar():
    covar_dask = dask_graph.compute()

[########################################] | 100% Completed | 97.45 s
